# Score Summary

In [1]:
from pathlib import Path
import altair as alt
import numpy as np
import pandas as pd

from espipeline.filemanager import FileManager

### Set up file manager

In [2]:
cluster = True  # working on cluster or local computer
prefix = Path('/mnt/team/msca') if cluster else Path('/Volumes/team_msca/')
i_folder = prefix / 'pub/archive/evidence-score/gbd2020-process'
o_folder = 'results'
fm = FileManager(i_folder, o_folder)

### Gather metadata from folders

In [3]:
metadata = []
for pair in fm.pairs:
    df = pd.read_csv(fm.pair_paths[pair] / 'risk_cause_metadata.csv')
    df_new = pd.read_csv(fm.pair_paths[pair] / 'new_risk_cause_metadata.csv')
    df["name"] = pair
    for col in ["score", "pub_bias"]:
        df[f"{col}_new"] = df_new[col]
    metadata.append(df)

pairs = pd.concat(metadata)
for col in ["rei_id", "cause_id"]:
    pairs[col].replace("unknown", -1, inplace=True)
pairs.to_csv(fm.o_path / 'score_summary.csv', index=False)

### Some pairs don't have ids

In [4]:
pairs[(pairs['rei_id'] == -1) | (pairs['cause_id'] == -1)]

,rei_id,cause_id,risk_lower,risk_upper,risk_unit,risk_type,score,pub_bias,name,score_new,pub_bias_new


### Some pairs don't have new scores yet

In [5]:
pairs[pairs['score_new'].isna()]

,rei_id,cause_id,risk_lower,risk_upper,risk_unit,risk_type,score,pub_bias,name,score_new,pub_bias_new


### Plot scores

Plots are interactive! Hover over data for tooltip and selections :)

In [6]:
selector = alt.selection_single(on='mouseover', nearest=True, empty='none')

chart = alt.Chart(pairs).mark_bar().transform_calculate(
    diff='datum.score_new - datum.score'
).add_selection(selector)

chart.encode(
    x='diff:Q',
    y=alt.Y('name', sort='-x'),
    color='pub_bias:N',
    stroke=alt.condition(selector, alt.value('red'), alt.value('white')),
    tooltip=[
        'name',
        alt.Tooltip('diff:Q', format='.4f'),
        alt.Tooltip('score', format='.4f'),
        alt.Tooltip('score_new', format='.4f')
    ]
)

alt.Chart(...)

In [ ]:
alt.Chart(pairs).mark_point().encode(
    x='score',
    y='score_new',
    color='pub_bias:N',
    tooltip=[
        'name',
        alt.Tooltip('score', format='.4f'),
        alt.Tooltip('score_new', format='.4f')
    ]
)